<a href="https://colab.research.google.com/github/Bishop1303/ML_FlightDelayClassifier/blob/master/ML_BucketingContVarFlights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Carica il drive con i dati:
from google.colab import drive
drive.mount('/content/drive')

# Getting the softwares:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark


# To use spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

# SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

#Bucketing continous data: departure time

Time of day data are a challenge with regression models.

The objective is to convert the flight departure times from numeric values between 0 (corresponding to 00:00) and 24 (corresponding to 24:00) to binned values. You'll then take those binned values and one-hot encode them.

#Flight duration model: Adding departure time

The objective is to include dummy variables in a regression model for flight duration.

The **km**, **org_dummy** and **depart_dummy** columns have been assembled into **features**, where:
* **km** is *index 0*, 
* **org_dummy** runs from *index 1 to 7* 
* **depart_dummy** from *index 8 to 14*  

Feature columns are:

* 0 — km
* 1 — ORD
* 2 — SFO
* 3 — JFK
* 4 — LGA
* 5 — SJC
* 6 — SMF
* 7 — TUS
* 8 — 00:00 to 03:00
* 9 — 03:00 to 06:00
* 10 — 06:00 to 09:00
* 11 — 09:00 to 12:00
* 12 — 12:00 to 15:00
* 13 — 15:00 to 18:00
* 14 — 18:00 to 21:00

In [4]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import round

# Read raw data to pyspark
flights = spark.read.csv("/content/drive/My Drive/flights.csv", inferSchema=True, header=True, mode='FAILFAST')
#flights.show()
#flights.printSchema()

### DATA PREPARATION ###

# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Remove records with missing 'delay' values or NA values
flights_none_missing = flights_drop_column.filter((flights_drop_column.delay.isNotNull()) & \
                                                  (flights_drop_column.delay != 'NA'))

# Test delay
flights_none_missing = flights_none_missing.withColumn('delay', flights_none_missing['delay'].cast('int'))

# Check on dataframe
#print('The Schema is: ')
#flights_none_missing.printSchema()
#print('=====================================================')
#print('Informative rows after dropping malformed: ',flights_none_missing.count())


# Conversion: 'mile' to 'km' and drop 'mile' column
flights_km = flights_none_missing.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')

# Creating 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check records
#flights_km.show(5)
#flights_km.printSchema()

idx_input_cols=['carrier', 'org']
idx_output_cols=['carrier_idx', 'org_idx']

# Create an indexer
indexer = StringIndexer(inputCols=idx_input_cols, outputCols=idx_output_cols)

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# Check result
#flights_indexed.show(5,False)

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights_indexed data
onehot = onehot.fit(flights_indexed)
flights_onehot = onehot.transform(flights_indexed)

# Check the results
#flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

# Create buckets at 3 hour intervals through the day
splits=[0,3,6,9,12,15,18,21,24]
buckets = Bucketizer(splits=splits, inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights_onehot)
#bucketed.select('depart','depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
#flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

# Create an assembler object
assembler = VectorAssembler(inputCols=['km','org_dummy','depart_dummy'], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_onehot)

# Check the resulting column
flights_assembled = flights_assembled.select('mon','dom','dow','carrier','org','depart','duration','delay','km',
                                             'org_idx','org_dummy','depart_bucket','depart_dummy','features')

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=7)

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
#predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
print('The RMSE of the model is:')
RegressionEvaluator(labelCol='duration').evaluate(predictions)



### Results ###

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print('Average time on ground at OGG for flights departing between 21:00 and 24:00',
      '{:,.1f}'.format(avg_eve_ogg), 'minutes')

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print('Average time on ground at OGG for flights departing between 00:00 and 03:00',
      '{:,.1f}'.format(avg_night_ogg), 'minutes')

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[8] + regression.coefficients[3]
print('Average time on ground at JFK for flights departing between 00:00 and 03:00',
      '{:,.1f}'.format(avg_night_jfk), 'minutes')

The RMSE of the model is:
Average time on ground at OGG for flights departing between 21:00 and 24:00 10.1 minutes
Average time on ground at OGG for flights departing between 00:00 and 03:00 -4.8 minutes
Average time on ground at JFK for flights departing between 00:00 and 03:00 47.0 minutes
